主要参考：https://zhuanlan.zhihu.com/p/610171544
使用BERT进行特征提取

In [2]:
#这里导入transformers的包，然后运行这段代码，如果安装成功的话会显示版本号
import transformers
transformers.__version__   

'4.19.4'

In [3]:
from transformers import BertTokenizer

#加载预训练字典和分词方法
tokenizer = BertTokenizer.from_pretrained(
    pretrained_model_name_or_path='bert-base-chinese',  # 可选，huggingface 中的预训练模型名称或路径，默认为 bert-base-chinese
    cache_dir=None,  # 将数据保存到的本地位置，使用cache_dir 可以指定文件下载位置
    force_download=False,   
)

sents = [
    '选择珠江花园的原因就是方便。',
    '笔记本的键盘确实爽。',
    '房间太小。其他的都一般。',
    '今天才知道这书还有第6卷,真有点郁闷.',
    '机器背面似乎被撕了张什么标签，残胶还在。',
]

tokenizer, sents

(PreTrainedTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}),
 ['选择珠江花园的原因就是方便。',
  '笔记本的键盘确实爽。',
  '房间太小。其他的都一般。',
  '今天才知道这书还有第6卷,真有点郁闷.',
  '机器背面似乎被撕了张什么标签，残胶还在。'])

In [4]:
#编码两个句子
out = tokenizer.encode(
    text=sents[0],
    text_pair=sents[1],  # 一次编码两个句子，若没有text_pair这个参数，就一次编码一个句子

    #当句子长度大于max_length时,截断
    truncation=True,

    #一律补pad到max_length长度
    padding='max_length',   # 少于max_length时就padding
    add_special_tokens=True,
    max_length=30,
    return_tensors=None,  # None表示不指定数据类型，默认返回list
)

print(out)

tokenizer.decode(out)

[101, 6848, 2885, 4403, 3736, 5709, 1736, 4638, 1333, 1728, 2218, 3221, 3175, 912, 511, 102, 5011, 6381, 3315, 4638, 7241, 4669, 4802, 2141, 4272, 511, 102, 0, 0, 0]


'[CLS] 选 择 珠 江 花 园 的 原 因 就 是 方 便 。 [SEP] 笔 记 本 的 键 盘 确 实 爽 。 [SEP] [PAD] [PAD] [PAD]'

主要的参数解释如下:  
text和text_pair:传入要编码的一句话或两句话文本。  
truncation:当句子长度大于max_length时,是否进行截断。True表示进行截断。  
padding:'max_length'表示句子长度小于max_length时,使用pad token在句子末尾补全到max_length长度。  
add_special_tokens:是否添加特殊字符,如[CLS]等。  
max_length:最大长度,若超过该长度则会截断。  
return_tensors:返回的数据类型,None表示返回数字list。  

编码的主要步骤是:  
使用词典,将文本tokenize成单词列表,比如['我','爱','北','京']  
添加特殊字符,如'[CLS]' ,'[SEP]'等。  
截断或padding句子到max_length长度。  
将单词映射为词典中的id。  
返回list或指定类型的张量。  
decode逆向操作,将编码后的id序列转换回文本。  

这种编码方式可以处理单句或双句的输入,并将其转换为模型可接受的标准长度序列输入。后续这种编码后的输入可直接用于模型训练或者预测。

In [5]:
#增强的编码函数
out = tokenizer.encode_plus(
    text=sents[0],
    text_pair=sents[1],

    #当句子长度大于max_length时,截断
    truncation=True,

    #一律补零到max_length长度
    padding='max_length',
    max_length=30,
    add_special_tokens=True,

    #可取值tensorflow,pytorch,numpy,默认值None为返回list
    return_tensors=None,

    #返回token_type_ids
    return_token_type_ids=True,

    #返回attention_mask
    return_attention_mask=True,

    #返回special_tokens_mask 特殊符号标识
    return_special_tokens_mask=True,

    #返回offset_mapping 标识每个词的起止位置,这个参数只能BertTokenizerFast使用
    #return_offsets_mapping=True,

    #返回length 标识长度
    return_length=True,
)

这个是对BertTokenizer的encode函数的增强,它不仅返回编码后的序列,还会返回一些额外的信息,比如token类型、attention mask等,这些信息在BERT模型的训练和预测中会用到。

主要的返回值说明:  
input_ids: 经过WordPiece编码的输入序列,就是encode的直接输出。  
token_type_ids: 区分第一个第二个句子的标志,用于表示是 encoder的输入是单句还是双句。  
attention_mask: 标记padding部分,让self-attention只关注非padding部分。  
special_tokens_mask: 标记special tokens的mask,比如[CLS]等。  
return_length: 返回输入的长度。  
所以encode_plus相比encode,提供了更全面的信息,包括输入序列、输入长度、标记不同部分的mask等,这些信息在BERT模型的训练和预测中都是必要的。  

调用返回的是字典类型,包含以上各个编码信息。我们可以直接拿这个字典喂给BERT模型,而不需要自己去计算mask、长度等信息。这样可以简化使用BERT的流程。

In [8]:
for k, v in out.items():
    print(k, ':', v)

tokenizer.decode(out['input_ids'])

input_ids : [101, 6848, 2885, 4403, 3736, 5709, 1736, 4638, 1333, 1728, 2218, 3221, 3175, 912, 511, 102, 5011, 6381, 3315, 4638, 7241, 4669, 4802, 2141, 4272, 511, 102, 0, 0, 0]
token_type_ids : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]
special_tokens_mask : [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
attention_mask : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]
length : 30


'[CLS] 选 择 珠 江 花 园 的 原 因 就 是 方 便 。 [SEP] 笔 记 本 的 键 盘 确 实 爽 。 [SEP] [PAD] [PAD] [PAD]'

In [6]:
#批量编码一个一个的句子
out = tokenizer.batch_encode_plus(
    batch_text_or_text_pairs=[sents[0], sents[1]],  # 批量编码，一次编码了两个句子(与增强的编码函数相比，就此处不同)

    #当句子长度大于max_length时,截断
    truncation=True,

    #一律补零到max_length长度
    padding='max_length',
    max_length=15,
    add_special_tokens=True,
    
    #可取值tf,pt,np,默认为返回list
    return_tensors=None,

    #返回token_type_ids
    return_token_type_ids=True,

    #返回attention_mask
    return_attention_mask=True,

    #返回special_tokens_mask 特殊符号标识
    return_special_tokens_mask=True,

    #返回offset_mapping 标识每个词的起止位置,这个参数只能BertTokenizerFast使用
    #return_offsets_mapping=True,

    #返回length 标识长度
    return_length=True,
)

In [10]:
for k, v in out.items():
    print(k, ':', v)

tokenizer.decode(out['input_ids'][0]), tokenizer.decode(out['input_ids'][1])


input_ids : [[101, 6848, 2885, 4403, 3736, 5709, 1736, 4638, 1333, 1728, 2218, 3221, 3175, 912, 102], [101, 5011, 6381, 3315, 4638, 7241, 4669, 4802, 2141, 4272, 511, 102, 0, 0, 0]]
token_type_ids : [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
special_tokens_mask : [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]]
length : [15, 12]
attention_mask : [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]]


('[CLS] 选 择 珠 江 花 园 的 原 因 就 是 方 便 [SEP]',
 '[CLS] 笔 记 本 的 键 盘 确 实 爽 。 [SEP] [PAD] [PAD] [PAD]')

In [7]:
#批量编码成对的句子
out = tokenizer.batch_encode_plus(
    batch_text_or_text_pairs=[(sents[0], sents[1]), (sents[2], sents[3])],   # tuple

    #当句子长度大于max_length时,截断
    truncation=True,

    #一律补零到max_length长度
    padding='max_length',
    max_length=30,
    add_special_tokens=True,

    #可取值tf,pt,np,默认为返回list
    return_tensors=None,

    #返回token_type_ids
    return_token_type_ids=True,

    #返回attention_mask
    return_attention_mask=True,

    #返回special_tokens_mask 特殊符号标识
    return_special_tokens_mask=True,

    #返回offset_mapping 标识每个词的起止位置,这个参数只能BertTokenizerFast使用
    #return_offsets_mapping=True,

    #返回length 标识长度
    return_length=True,
)


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


In [12]:
for k, v in out.items():
    print(k, ':', v)

tokenizer.decode(out['input_ids'][0])

input_ids : [[101, 6848, 2885, 4403, 3736, 5709, 1736, 4638, 1333, 1728, 2218, 3221, 3175, 912, 511, 102, 5011, 6381, 3315, 4638, 7241, 4669, 4802, 2141, 4272, 511, 102, 0, 0, 0], [101, 2791, 7313, 1922, 2207, 511, 1071, 800, 4638, 6963, 671, 5663, 511, 102, 791, 1921, 2798, 4761, 6887, 6821, 741, 6820, 3300, 5018, 127, 1318, 117, 4696, 3300, 102]]
token_type_ids : [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
special_tokens_mask : [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
length : [27, 30]
attention_mask : [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]


'[CLS] 选 择 珠 江 花 园 的 原 因 就 是 方 便 。 [SEP] 笔 记 本 的 键 盘 确 实 爽 。 [SEP] [PAD] [PAD] [PAD]'

In [13]:
#获取字典
zidian = tokenizer.get_vocab()

type(zidian), len(zidian), '月光' in zidian,   # (dict, 21128, False)

(dict, 21128, False)

这里我们调用了get_vocab()方法来获取分词器的词典。词典是一个字典类型,键是词,值是词对应的数字索引。

我们可以看到:  

词典是dict类型   
词典大小是21128个词  
词典中不包含'月光'这个词  
这验证了BERT使用的是WordPiece算法进行分词,词典中包含字母、词汇和词组,但不会包含所有汉语词汇。  

通过获取词典我们可以了解到分词器的词汇量和词汇覆盖情况,也可以直接查询某个词是否在词典中。这对分析BERT语言表示能力以及优化词典非常有帮助。

In [9]:
#添加新词
tokenizer.add_tokens(new_tokens=['月光', '希望'])

#添加新符号
tokenizer.add_special_tokens({'eos_token': '[EOS]'})   # End Of Sentence

zidian = tokenizer.get_vocab()

type(zidian), len(zidian), zidian['月光'], zidian['[EOS]'],zidian['希望']   # (dict, 21131, 21128, 21130)

(dict, 21131, 21128, 21130, 21129)

In [15]:
#编码新添加的词
out = tokenizer.encode(
    text='月光的新希望[EOS]',
    text_pair=None,

    #当句子长度大于max_length时,截断
    truncation=True,

    #一律补pad到max_length长度
    padding='max_length',
    add_special_tokens=True,
    max_length=8,
    
    return_tensors=None,
)

print(out)

tokenizer.decode(out)

[101, 21128, 4638, 3173, 21129, 21130, 102, 0]


'[CLS] 月光 的 新 希望 [EOS] [SEP] [PAD]'

In [12]:
#ChnSentiCorp数据集包含正面和负面评论句子,可以用来进行中文文本的情感分析。
#https://github.com/brightmart/nlp_chinese_corpus

from datasets import load_dataset

#加载数据
dataset = load_dataset(path='lansinuote/ChnSentiCorp')
dataset

Found cached dataset parquet (/Users/apple/.cache/huggingface/datasets/lansinuote___parquet/lansinuote--ChnSentiCorp-eaea6a9750cb0fe7/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9600
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
})

In [17]:
#保存数据集到磁盘
dataset.save_to_disk(dataset_dict_path='./data/ChnSentiCorp')

Saving the dataset (0/1 shards):   0%|          | 0/9600 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1200 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1200 [00:00<?, ? examples/s]

In [18]:
#取出训练集
dataset = dataset['train']
#查看一个数据
dataset[0]

{'text': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般',
 'label': 1}

In [13]:
from datasets import list_metrics

#列出评价指标
metrics_list = list_metrics()
len(metrics_list), metrics_list

/var/folders/j9/ryqgqp6n03schy65y0b0g4sm0000gn/T/ipykernel_42992/1035245866.py:4: FutureWarning: list_metrics is deprecated and will be removed in the next major version of datasets. Use 'evaluate.list_evaluation_modules' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metrics_list = list_metrics()


(119,
 ['accuracy',
  'bertscore',
  'bleu',
  'bleurt',
  'brier_score',
  'cer',
  'character',
  'charcut_mt',
  'chrf',
  'code_eval',
  'comet',
  'competition_math',
  'coval',
  'cuad',
  'exact_match',
  'f1',
  'frugalscore',
  'glue',
  'google_bleu',
  'indic_glue',
  'mae',
  'mahalanobis',
  'mape',
  'mase',
  'matthews_correlation',
  'mauve',
  'mean_iou',
  'meteor',
  'mse',
  'nist_mt',
  'pearsonr',
  'perplexity',
  'poseval',
  'precision',
  'r_squared',
  'recall',
  'rl_reliability',
  'roc_auc',
  'rouge',
  'sacrebleu',
  'sari',
  'seqeval',
  'smape',
  'spearmanr',
  'squad',
  'squad_v2',
  'super_glue',
  'ter',
  'trec_eval',
  'wer',
  'wiki_split',
  'xnli',
  'xtreme_s',
  'AlhitawiMohammed22/CER_Hu-Evaluation-Metrics',
  'BucketHeadP65/confusion_matrix',
  'BucketHeadP65/roc_curve',
  'Drunper/metrica_tesi',
  'Felipehonorato/eer',
  'He-Xingwei/sari_metric',
  'JP-SystemsX/nDCG',
  'Josh98/nl2bash_m',
  'Kyle1668/squad',
  'Muennighoff/code_eval',


这里使用datasets库的list_metrics函数列出了所有内置的评价指标。这样就不用自己找函数定义评价指标或者计算评价指标了。  
使用list_metrics可以快速查看datasets当前支持的所有评价指标。

In [14]:
from datasets import load_metric

#加载一个评价指标
metric = load_metric('glue', 'mrpc')   # MRPC(The Microsoft Research Paraphrase Corpus，微软研究院释义语料库)
print(metric.inputs_description)

/var/folders/j9/ryqgqp6n03schy65y0b0g4sm0000gn/T/ipykernel_42992/1543994463.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', 'mrpc')   # MRPC(The Microsoft Research Paraphrase Corpus，微软研究院释义语料库)



Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(results)
    {'accuracy': 1.0}

    >>> glue_metric = datasets.load_metric('glue', 'mrpc')  # 'mrpc' or 'qqp'
    >>> references = [0, 1]

In [21]:
#计算一个评价指标
predictions = [0, 1, 0]
references = [0, 1, 1]

final_score = metric.compute(predictions=predictions, references=references)
final_score

{'accuracy': 0.6666666666666666, 'f1': 0.6666666666666666}

In [15]:
import torch
from datasets import load_dataset

#定义数据集
class Dataset(torch.utils.data.Dataset):
    def __init__(self, split):
        # 加载数据。ChnSentiCorp为消费评价数据集，分好评和差评
        self.dataset = load_dataset(path='lansinuote/ChnSentiCorp', split=split)  # Available splits: ['test', 'train', 'validation']

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset[i]['text']
        label = self.dataset[i]['label']
        return text, label

dataset = Dataset('train')
len(dataset), dataset[0]  # 训练集有9600句话；dataset[0]表示第1句话，前面是text，后面是label，1表示差评

Found cached dataset parquet (/Users/apple/.cache/huggingface/datasets/lansinuote___parquet/lansinuote--ChnSentiCorp-eaea6a9750cb0fe7/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


(9600,
 ('选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般',
  1))

In [16]:
from transformers import BertTokenizer

#加载字典和分词工具，即tokenizer
token = BertTokenizer.from_pretrained('bert-base-chinese')  # 要跟预训练模型相匹配
token

PreTrainedTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [17]:
# 定义批处理函数
def collate_fn(data):
    sents = [i[0] for i in data]#提取文本
    labels = [i[1] for i in data]#提取标签

    #编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,   # 当句子长度大于max_length时，截断
                                   padding='max_length',   # 一律补0到max_length长度
                                   max_length=500,
                                   return_tensors='pt',   # 返回pytorch类型的tensor
                                   return_length=True)   # 返回length，标识长度

    input_ids = data['input_ids']    # input_ids:编码之后的数字
    attention_mask = data['attention_mask']     # attention_mask:补零的位置是0,其他位置是1
    token_type_ids = data['token_type_ids']   # 第一个句子和特殊符号的位置是0，第二个句子的位置是1(包括第二个句子后的[SEP])
    labels = torch.LongTensor(labels)

    return input_ids, attention_mask, token_type_ids, labels

In [18]:
#数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

# 查看数据样例
for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):
    break

print(len(loader))  # 600 = 9600/16
input_ids.shape, attention_mask.shape, token_type_ids.shape, labels   # 500表示句子最大长度为500


600


(torch.Size([16, 500]),
 torch.Size([16, 500]),
 torch.Size([16, 500]),
 tensor([1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0]))

定义了一个DataLoader,参数包括:  
dataset:要加载的数据集  
batch_size:批大小为16  
collate_fn:使用之前定义好的collate函数进行批处理  
shuffle:打乱数据  
drop_last:去掉最后不完整的batch  
遍历loader,查看一个batch的数据样例  

打印loader的长度,以及一个batch中各个tensor的shape

In [19]:
from transformers import BertModel

#加载预训练模型
pretrained = BertModel.from_pretrained('bert-base-chinese')
print(pretrained)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [20]:
#不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)  # 这里不使用fine-tuning，直接把预训练模型的参数冻结住，只训练下游任务模型，对预训练模型本身的参数不调整

这段代码的目的是冻结预训练语言模型(如BERT)的参数,使其在训练下游任务时不更新。

主要步骤是:

遍历预训练模型pretrained中的所有参数param。

对每个参数调用requires_grad_(False),将requires_grad标志置为False。

这样在训练时计算图就不会跟踪这些参数的梯度,实现了冻结预训练参数的效果。

这个技巧通常用在使用预训练语言模型做下游任务的时候,先冻结预训练模型,只训练新增的参数,然后再解冻后整体微调。

它可以防止预训练参数在下游任务训练初期被破坏,并且加速收敛。



In [21]:
#模型试算
out = pretrained(input_ids=input_ids,
           attention_mask=attention_mask,
           token_type_ids=token_type_ids)

out.last_hidden_state.shape   # [batch_size, 数据分词的长度(每一句话编码成500个词的长度), 词编码的维度(即每一个词编码成一个768维的向量)]


torch.Size([16, 500, 768])

这里使用了预训练语言模型(如BERT)对输入进行了前向计算,并打印出输出tensor的shape。

主要的步骤是:  
将经过分词和编码后的input作为输入,包括input_ids,attention_mask,token_type_ids。  
把这些输入传给预训练模型pretrained,调用模型执行前向计算。  
从输出out中取出最后一层的隐状态last_hidden_state。  
打印出last_hidden_state的shape。  

从打印的shape可以看出:  
第一个维度是batch size,即一次输入的样本数量。  
第二个维度是输入序列长度,经过分词和truncation/padding后都处理成了固定长度。  
第三个维度是词向量维度,即BERT模型对每个词编码成的向量表示维度。  
所以输出tensor的形状反映了模型处理批量序列输入的结果。

每次向BERT等预训练语言模型提供输入时,都需要按照这个格式提供。  
具体来说,输入对应的模型部分是:  
input_ids:对应embedding层的输入,表示单词的索引信息。  
attention_mask:对应self-attention层的mask输入,指示哪些词是padding,哪些词是有效词。  
token_type_ids:对应第一句和第二句的区分,让模型学习句子关系。  
这三个输入分别提供了输入序列、标记和句子关系信息。它们共同定义了模型的文本输入。

模型内部的计算流程是:  
输入层根据input_ids查表获取词向量表示;  
多层Transformer编码器结构捕获上下文信息;  
最终输出包含句子级语义信息的词向量。

In [30]:
#定义下游任务模型
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768, 2)  # 单层网络模型，只包括了一个fc的神经网络

    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,   # 先拿预训练模型来做一个计算，抽取数据当中的特征
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids)

        # 把抽取出来的特征放到全连接网络中运算，且特征的结果只需要第0个词的特征(跟bert模型的设计方式有关。对句子的情感分类，只需要拿特征中的第0个词来进行分类就可以了)
        out = self.fc(out.last_hidden_state[:, 0])   # torch.Size([16, 768])
        
        # 将softmax函数应用于一个n维输入张量，对其进行缩放，使n维输出张量的元素位于[0,1]范围内，总和为1
        out = out.softmax(dim=1)  

        return out

model = Model()

model(input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids=token_type_ids).shape    # torch.Size([16, 2])


torch.Size([16, 2])

这个定义的这个下游任务模型实现了基于BERT的文本分类,主要步骤如下:
定义Model类,继承torch.nn.Module。  
初始化时定义了一个线性层fc,输入是BERT输出维度768,输出是2分类。  
forward函数中,首先使用torch.no_grad()不计算梯度地运行BERT获取特征。  
然后取最后一层隐状态的第一个token作为分类特征,输入到fc层。  
对fc层输出使用softmax获得归一化的预测分布。  
这样即实现了利用BERT提取特征,然后接一个简单的线性分类器来进行文本分类。

取最后一层第一个token作为句子表示是BERT特有的做法

In [34]:
from transformers import AdamW   # 优化器，即 Adam + Weight decay(自适应梯度方法)

#训练下游任务模型
optimizer = AdamW(model.parameters(), lr=5e-4)
criterion = torch.nn.CrossEntropyLoss()  # Pytorch计算交叉熵误差的函数自带softmax，故训练时模型里不要添加softmax

model.train()

for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):
    out = model(input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)

    loss = criterion(out, labels)  # 输出跟真实的labels计算loss
    loss.backward()   # 调用反向传播得到每个要更新参数的梯度
    optimizer.step()  # 每个参数根据上一步得到的梯度进行优化
    optimizer.zero_grad()  # 把上一步训练的每个参数的梯度清零

    if i % 5 == 0:
        out = out.argmax(dim=1)
        accuracy = (out == labels).sum().item() / len(labels)

        print(i, loss.item(), accuracy)
        if i == 300:  # 只训练300个轮次，没有把全量的数据(len(loader)= 600 = 9600/16)全部训练一遍
            break

/Users/apple/miniforge3/envs/env_tf/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0 0.7090100049972534 0.375
5 0.6451802253723145 0.75
10 0.6374646425247192 0.6875
15 0.6967511773109436 0.5625
20 0.6140915155410767 0.75
25 0.5840007066726685 0.75
30 0.5491213202476501 0.9375
35 0.6162095665931702 0.75
40 0.5880313515663147 0.625
45 0.5858539938926697 0.75
50 0.5374869704246521 0.875
55 0.5708539485931396 0.75
60 0.533920407295227 0.8125
65 0.5271445512771606 0.875
70 0.49780941009521484 0.875
75 0.5622323155403137 0.875
80 0.5095774531364441 0.875
85 0.47063907980918884 0.875
90 0.4942939281463623 0.8125
95 0.4704240560531616 0.9375
100 0.4980355203151703 0.8125
105 0.4693685472011566 0.9375
110 0.4152890145778656 1.0
115 0.4286099970340729 0.9375
120 0.4034077227115631 1.0
125 0.44301509857177734 1.0
130 0.5175010561943054 0.75
135 0.4954994022846222 0.875
140 0.45595037937164307 0.9375
145 0.4319436550140381 0.9375
150 0.5459808111190796 0.875
155 0.47220784425735474 0.875
160 0.3745757043361664 0.9375
165 0.48836103081703186 0.8125
170 0.5114022493362427 0.875
17

In [35]:
#测试
def test():
    model.eval()
    
    correct = 0
    total = 0

    loader_test = torch.utils.data.DataLoader(dataset=Dataset('validation'),
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader_test):

        if i == 5:
            break   # 只测试前5个

        print(i)

        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)

        out = out.argmax(dim=1)
        correct += (out == labels).sum().item()
        total += len(labels)

    print(correct / total)  # 测试集的正确率


test()

Found cached dataset parquet (/Users/apple/.cache/huggingface/datasets/lansinuote___parquet/lansinuote--ChnSentiCorp-eaea6a9750cb0fe7/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


0
1
2
3
4
0.85625
